# Topological feature extraction from (un)directed graphs: `VietorisRipsPersistence`, `SparseRipsPersistence` and `FlagserPersistence`

`giotto-tda` allows the extraction of topological features from undirected or directed graphs represented as adjacency matrices, via the following transformers:

- [`VietorisRipsPersistence`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/homology/gtda.homology.VietorisRipsPersistence.html) and [`SparseRipsPersistence`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/homology/gtda.homology.VietorisRipsPersistence.html) initialized with `metric="precomputed"`, for undirected graphs;
- [`FlagserPersistence`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/homology/gtda.homology.VietorisRipsPersistence.html) initialized with `directed=True`, for directed graphs (and with `directed=False` for undirected ones).

In this notebook, we build some basic intuition on how these methods are able to capture structures and patterns from such graphs. We will focus on the multi-scale nature of the information contained in the final outputs ("persistence diagrams"), as well as on the differences between the undirected and directed cases. Another important point is that, although adjacency matrices of sparsely connected and even unweighted graphs can be passed directly to these transformers, they will always be interpreted as **weighted adjacency matrices** according to some not completely standard conventions. We will highlight these below.

The mathematical technologies used under the hood are various flavours of "persistent homology" (as is also the case for [`EuclideanCechPersistence`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/homology/gtda.homology.EuclideanCechPersistence.html) and [`CubicalPersistence`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/homology/gtda.homology.CubicalPersistence.html)). If you are interested in the details, you can start from the [theory glossary](https://giotto-ai.github.io/gtda-docs/latest/theory/glossary.html) and references therein.

If you are looking at a static version of this notebook and would like to run its contents, head over to [github](https://github.com/giotto-ai/giotto-tda/blob/master/examples/persistent_homology_graphs.ipynb).


## See also

- [Topological feature extraction using `VietorisRipsPersistence` and `PersistenceEntropy`](https://giotto-ai.github.io/gtda-docs/latest/notebooks/vietoris_rips_quickstart.html) which treats the "special case" of point clouds (see below).
- [Plotting in `giotto-tda`](https://giotto-ai.github.io/gtda-docs/latest/notebooks/plotting_api.html), particularly Section 1.2 (as above, treats the case of point clouds).
- [Case study: Classification of shapes](https://giotto-ai.github.io/gtda-docs/latest/notebooks/classifying_shapes.html) (a more advanced example).

**License: AGPLv3**

## Import libraries

In [ ]:
import numpy as np
from numpy.random import default_rng
rng = default_rng(42)  # Create a random number generator

from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix

from gtda.plotting import plot_diagram
from gtda.graphs import GraphGeodesicDistance
from gtda.homology import VietorisRipsPersistence, SparseRipsPersistence, FlagserPersistence

from igraph import Graph

from IPython.display import SVG, display

## Undirected graphs – `VietorisRipsPersistence` and `SparseRipsPersistence`

### Fully connected and weighted

We begin with the case of fully connected and weighted (FCW) undirected graphs. The short version of the story is that if you have a collection `X` of weighted adjacency matrices (which could be sparse or dense, and possibly upper-triangular), you can instantiate tranformers of class `VietorisRipsPersistence` or `SparseRipsPersistence` by setting the parameter `metric` as `"precomputed"`, and then call `fit_transform` on `X` to obtain the corresponding collection of *persistence diagrams* (see below for an explanation).

In [ ]:
# Start creating a single 10 x 10 weighted adjacency matrix (of a FCW graph)
x = rng.random((10, 10))
# Fill the diagonal with zeros (not always necessary, see below)
np.fill_diagonal(x, 0)

# Create a trivial collection of weighted adjacency matrices, containing x only
X = [x]  # Alternative as a 3D array: X = x.reshape((1, **x.shape))

# Instantiate transformers with `metric="precomputed"` and default remaining parameters
VR, SR = VietorisRipsPersistence(metric="precomputed"), SparseRipsPersistence(metric="precomputed", epsilon=0.1)

# Compute persistence diagrams corresponding to each entry in X
X_vr, X_sr = VR.fit_transform(X), SR.fit_transform(X)

print(f"X_vr.shape: {X_vr.shape} (16 topological features)\nX_sr.shape: {X_sr.shape} (15 topological features)")

`SparseRipsPersistence` actually implements an approximate scheme for computing the same topological quantities as `VietorisRipsPersistence`, with possibly very large speedups when the inputs have large sizes. We can plot the two results:

In [ ]:
sample = 0
for transformer, output in [(VR, X_vr), (SR, X_sr)]:
    title = f"Persistence diagram computed from FCW graph {sample}<br>using {transformer.__class__.__name__}"
    VR.plot(output,
            sample=sample,
            plotly_params={"layout": {"title": title}}).show()

... Not as similar as we might have hoped! This is to be expected with such a small input matrix and with the default value of `SparseRipsPersistence`'s `epsilon` parameter. But instead of trying to change things, we will leave `SparseRipsPersistence` aside for the rest of the notebook and focus on the meaning of the output in the case of `VietorisRipsPersistence`.

### Non-fully connected weighted graphs

In `giotto-tda`, a non-fully connected weighted graph can be represented by an adjacency matrix in one of two possible forms:
- a dense square array with `np.inf` in position ij if the edge between vertex i and vertex j is absent.
- a sparse matrix in which the non-stored edge weights represent absent edges.

**Important notes**
- A `0` in a dense array, or an explicitly stored `0` in a sparse matrix, does *not* denote an absent edge. It denotes an edge with weight 0, which, in a sense, means the complete opposite! See the section ***Understanding the computation*** below.
- Dense Boolean arrays are first converted to numerical ones and then interpreted as adjacency matrices of FCW graphs. `False` values therefore should not be used to represent absent edges.

### Understanding the computation 

To understand what these persistence diagrams are telling us about the input weighted graphs, we briefly explain the **clique complex (or flag complex) filtration** procedure underlying the computations in `VietorisRipsPersistence` when `metric="precomputed"`, via an example.

Let us start with a special case of a weighted graph with adjacency matrix as follows:
- the diagonal entries ("vertex weights") are all zero;
- all off-diagonal entries (edge weights) are non-negative;
- some edge weights are infinite (or very very large).

We can lay such a graph on the plane to visualise it, drawing only the finite edges:
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/weighted_graph.png)

The procedure can be explained as follows: we let a parameter $\varepsilon$ start at 0, and as we increase it all the way to infinity we keep considering the instantaneous subgraphs made of a) all the vertices in the original graph, and b) those edges whose weight is less than or equal to the current $\varepsilon$. We also promote these subgraphs to more general structures called **(simplicial) complexes** that, alongside vertices and edges, also possess $k$**-simplices**, i.e. selected subsets of $k + 1$ vertices (a 2-simplex is an abstract "triangle", a 3-simplex an abstract "tetrahedron", etc). Our criterion is this: for each integer $k \geq 2$, all $(k + 1)$-cliques in each instantaneous subgraph are declared to be the $k$-simplices of the subgraph's associated complex. By definition, the $0$-simplices are the vertices and the $1$-simplices are the available edges.

As $\varepsilon$ increases from 0 (included) to infinity, we record the following information:
1. How many new **connected components** are created because of the appearance of vertices (in this example, all vertices "appear" in one go at $\varepsilon = 0$, by definition!), or merge because of the appearance of new edges.
2. How many new 1-dimensional "holes", 2-dimensional "cavities", or more generally $d$-dimensional **voids** are created in the instantaneous complex. A hole, cavity, or $d$-dimensional void is such only if there is no collection of "triangles", "tetrahedra", or $(d + 1)$-simplices which the void is the "boundary" of. *Note*: Although the edges of a triangle *alone* "surround a hole", these cannot occur in our particular construction because the "filling" triangle is also declared present in the complex when all its edges are.
3. How many $d$-dimensional voids which were present at earlier values of $\epsilon$ are "filled" by $(d + 1)$-simplices which just appear.

This process of recording the full topological history of the graph across all edge weights is called (Vietoris-Rips) **persistent homology**!

Let us start at $\varepsilon = 0$: Some edges had zero weight in our graph, so they already appear!
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_0_small.png)
There are 9 connected components, and nothing much else.

Up to and including $\varepsilon = 2$, a few more edges are added which make some of the connected components merge together but do not create any hole, triangles, or higher cliques. Let us look at $\varepsilon = 3$:
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_3_small.png)
The newly arrived edges reduce the number of connected components further, but more interestingly they create a 1D hole!

As an example of a "higher"-simplex, at $\varepsilon = 4$ we get our first triangle:
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_4_small.png)

At $\varepsilon = 5$, our 1D hole is filled:
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_5_small.png)

At $\varepsilon = 8$, two new 1D holes appear:
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_8_small.png)

Finally, at $\varepsilon = 9$, some more connected components merge, but no new voids are either created or destroyed:
![](https://raw.githubusercontent.com/ulupo/giotto-tda/examples/flagser_notebook/examples/images/clique_complex_9_small.png)

We can stop as we have reached the maximum value of $\varepsilon$, beyond which nothing will change: there is only one connected component left, but there are also two 1D holes which will never get filled.

`VietorisRipsPersistence(metric="precomputed")` on the original graph's adjacency matrix would return the result of the computation like this:

In [ ]:
persistence_diagrams = np.array([[[0., 1., 0],
                                  [0., 2., 0],
                                  [0., 2., 0],
                                  [0., 3., 0],
                                  [0., 4., 0],
                                  [0., 5., 0],
                                  [0., 6., 0],
                                  [0., 7., 0],
                                  [3., 5., 1],
                                  [8., np.inf, 1],
                                  [8., np.inf, 1]]])

It is a 3D array simply because `giotto-tda` processes inputs in batches, as we saw above, and we imagine we passed a list/array of graphs the first of which is the one in the example. So `persistence_diagrams[0]` contains the result of our persistent homology computation, and it is a 2D array interpreted as follows:
- The first and second columns contains the $\varepsilon$ values corresponding to the "birth" and "death" (respectively) of a single topological feature.
- The third column tells us what "**homology dimension**" this feature is in: connected components are 0-dimensional features, and $d$-dimensional voids are $d$-dimensional features.

If we make one scatter plot per available homology dimension, and plot births and deaths as x- and y-coordinates of points in 2D, we end up with the representation known as a **persistence diagram**:

In [ ]:
plot_diagram(persistence_diagrams[0])

*Small aside*: You would be correct to expect an additional row `[0, np.inf, 0]` representing one connected component which lives forever. By convention, since such a row would always be present under this construction and hence give no useful information, all transformers discussed in this notebook remove this feature from the output.

#### Advanced discussion: Non-zero vertex weights and negative edge weights

Although we introcuded the simplifying assumptions that the diagonal entries of the input adjacency matrix is zero, and that all edge weights are non-negative, for the procedure to make sense we need a lot less. Namely:
- The diagonal entry corresponding to a vertex is always interpreted as the value of the parameter $\varepsilon$ at which that vertex "appears". Making all these entries equal to zero means, as in the example above, that all vertices appear simultaneously at $\varepsilon = 0$. Generally however, different vertices can be declared to "appear" at different values, and even at negative ones.
- The only constraint on each edge weight is that it should be no less than the "vertex weight" of either of its boundary vertices.

As a simple example, subtracting a constant from *all* entries of an adjacency matrix has the effect of shifting all birth and death values by the same constant.

### The "special case" of point clouds

The use of `VietorisRipsPersistence` to compute multi-scale topological features of concrete point clouds in Euclidean space is covered briefly in Section 1.2 of [Plotting in `giotto-tda`](https://giotto-ai.github.io/gtda-docs/latest/notebooks/plotting_api.html), and more in-depth in [Case study: Classification of shapes](https://giotto-ai.github.io/gtda-docs/latest/notebooks/classifying_shapes.html) and in [Can two-dimensional topological voids exist in two dimensions?](https://giotto-ai.github.io/gtda-docs/latest/notebooks/voids_on_the_plane.html)

The Vietoris-Rips procedure for point clouds is often depicted as a process of growing balls of ever increasing radius $r$ around each point, and drawing edges between two points whenever their two respective $r$-balls touch for the first time. Just as in our clique complex construction above, cliques present at radius $r$ are declared to be higher-dimensional simplices in the instantaneous complex:
![](https://miro.medium.com/max/1200/1*w3BiQI1OX93KXcezctRQTQ.gif "segment")
And just as in the case of weighted graphs, we record the appearance/disappearance of connected components and voids as we keep increasing $r$.

The case of point clouds can actually be thought of as a special case of the case of FCW graphs. Namely, if:
1. we regard each point in the cloud as an abstract vertex in a graph,
2. we compute the square matrix of pairwise (Euclidean or other) distances between points in the cloud, and
3. we run the procedure explained above with $\varepsilon$ defined as $2r$,
then we compute exactly the "topological summary" of the point cloud.

So, in `giotto-tda`, we can do:

In [ ]:
# 1 point cloud with 20 points in 5 dimensions
point_cloud = rng.random((20, 5))
# Corresponding matrix of Euclidean pairwise distances
pairwise_distances = squareform(pdist(point_cloud))

# Default parameter for `metric` is "euclidean"
X_vr_pc = VietorisRipsPersistence().fit_transform([point_cloud])

X_vr_graph = VietorisRipsPersistence(metric="precomputed").fit_transform([pairwise_distances])

assert np.array_equal(X_vr_pc, X_vr_graph)

### Unweighted graphs and chaining with `GraphGeodesicDistance`

What if, as is the case in many applications, our graphs have sparse connections and are unweighted?

In `giotto-tda`, there are two possibilities:
1. One can encode the graphs as adjacency matrices of non-fully connected weighted graphs, where all weights corresponding to edges which are present are equal to `1.` (or any other positive constant). See section ***Non-fully connected weighted graphs*** above for the different encoding conventions for sparse and dense matrices.
2. One can preprocess the unweighted graph via [`GraphGeodesicDistance`](https://giotto-ai.github.io/gtda-docs/latest/modules/generated/graphs/processing/gtda.graphs.GraphGeodesicDistance.html) to obtain a FCW graph where edge ij has as weight the length of the shortest path from vertex i to vertex j (and `np.inf` if no path exists between the two vertices in the original graph).

We now explore the difference between the two approaches in the simple example of a circle graph.

In [ ]:
# Helper functoin -- directed circles will be needed later
def make_circle_adjacency(n_vertices, directed=False):
    weights = np.ones(n_vertices)
    rows = np.arange(n_vertices)
    columns = np.arange(1, n_vertices + 1) % n_vertices
    directed_adjacency = csr_matrix((weights, (rows, columns)))
    if not directed:
        return directed_adjacency + directed_adjacency.T
    return directed_adjacency

n_vertices = 10
undirected_circle = make_circle_adjacency(n_vertices)

We can produce an SVG of the circle using `python-igraph`, and display it.

In [ ]:
row, col = undirected_circle.nonzero()
graph = Graph(n=n_vertices, edges=list(zip(row, col)), directed=False)
fname = "undirected_circle.svg"
graph.write_svg(fname)
display(SVG(filename=fname))

## Note: if `pycairo` is installed, we can avoid dumping files and draw directly in the notebook
# from igraph import plot
# plot(graph)

Approach 1 means passing the graph as is:

In [ ]:
VietorisRipsPersistence(metric="precomputed").fit_transform_plot([undirected_circle]);

The circular nature has been captured by the single point in homology dimension 1 ("H_1") which is born at 1 and lives forever.

Compare with what we observe when preprocessing first with `GraphGeodesicDistance`:

In [ ]:
X_ggd = GraphGeodesicDistance(directed=False, unweighted=True).fit_transform([undirected_circle])
VietorisRipsPersistence(metric="precomputed").fit_transform_plot(X_ggd);

There is still a "long-lived" topological feature in dimension 1, but this time its death value is finite. This is because, at some point, we have enough triangles to completely fill the 1D hole. Indeed, when the number of vertices/edges in the circle is large, the death value is around one third of this number. So, relative to the procedure without `GraphGeodesicDistance`, the death value now gives additional information about the *size* of the circle graph!

Suppose our graph contains two disconnected circles of different sizes:

In [ ]:
n_vertices_small, n_vertices_large = n_vertices, 2 * n_vertices
undirected_circle_small = make_circle_adjacency(n_vertices_small)
undirected_circle_large = make_circle_adjacency(n_vertices_large)
row_small, col_small = undirected_circle_small.nonzero()
row_large, col_large = undirected_circle_large.nonzero()
row = np.concatenate([row_small, row_large + n_vertices])
col = np.concatenate([col_small, col_large + n_vertices])
data = np.concatenate([undirected_circle_small.data, undirected_circle_large.data])
two_undirected_circles = csr_matrix((data, (row, col)))

graph = Graph(n=n_vertices_small + n_vertices_large, edges=list(zip(row, col)), directed=False)
fname = "two_undirected_circles.svg"
graph.write_svg(fname)
display(SVG(filename=fname))

Again, the first procedure just says "there are two one-dimensional holes".

In [ ]:
VietorisRipsPersistence(metric="precomputed").fit_transform_plot([two_undirected_circles]);

The second procedure is again much more informative, yielding a persistence diagram with two points in homology dimension 1 with different finite deaths, each corresponding to one of the two circles:

In [ ]:
X_ggd = GraphGeodesicDistance(directed=False, unweighted=True).fit_transform([two_undirected_circles])
VietorisRipsPersistence(metric="precomputed").fit_transform_plot(X_ggd);

## Directed graphs and `FlagserPersistence`

Together with the companion package `pyflagser` ([source code](https://github.com/giotto-ai/pyflagser), [API reference](https://docs-pyflagser.giotto.ai/)), `giotto-tda` allows the extraction of topological features from *directed* (weighted or unweighted) graphs. 

In [ ]:
homology_dimensions = [0, 1, 2]

In [ ]:
row, col = directed_circle.nonzero()
graph = Graph(n=n_vertices, edges=list(zip(row, col)), directed=True)
fname = "graph.svg"
graph.write_svg(fname)
display(SVG(filename=fname))

In [ ]:
directed_circle_gd = GraphGeodesicDistance(directed=True).fit_transform([directed_circle])
undirected_circle_gd = GraphGeodesicDistance(directed=False).fit_transform([directed_circle])

In [ ]:
FP = FlagserPersistence(directed=True, homology_dimensions=homology_dimensions)
FP.fit_transform_plot(directed_circle_gd);

In [ ]:
FP = FlagserPersistence(directed=False, homology_dimensions=homology_dimensions)
FP.fit_transform_plot(undirected_circle_gd);

# # Alternative
# VR = VietorisRipsPersistence(metric="precomputed", homology_dimensions=homology_dimensions)
# VR.fit_transform_plot(undirected_circle_gd);

In [ ]:
rows_flipped = np.concatenate([row[::2], col[1::2]])
columns_flipped = np.concatenate([col[::2], row[1::2]])
weights = np.ones(n_vertices)
directed_circle_flipped = csr_matrix((weights, (rows_flipped, columns_flipped)), shape=(n_vertices, n_vertices))

graph = Graph(n=n_vertices, edges=list(zip(rows_flipped, columns_flipped)), directed=True)
fname = "graph.svg"
graph.write_svg(fname)
display(SVG(filename=fname))

In [ ]:
directed_circle_flipped_gd = GraphGeodesicDistance(directed=True).fit_transform([directed_circle_flipped])
FP = FlagserPersistence(directed=True, homology_dimensions=homology_dimensions)
FP.fit_transform_plot(directed_circle_flipped_gd);

In [ ]:
rows_fixed_point = np.concatenate([row[:n_vertices // 2], col[n_vertices // 2:]])
columns_fixed_point = np.concatenate([col[:n_vertices // 2], row[n_vertices // 2:]])
weights = np.ones(n_vertices)
directed_circle_fixed_point = csr_matrix((weights, (rows_fixed_point, columns_fixed_point)), shape=(n_vertices, n_vertices))

graph = Graph(n=n_vertices, edges=list(zip(rows_fixed_point, columns_fixed_point)), directed=True)
fname = "graph.svg"
graph.write_svg(fname)
display(SVG(filename=fname))

In [ ]:
directed_circle_fixed_point_gd = GraphGeodesicDistance(directed=True).fit_transform([directed_circle_fixed_point])
FP = FlagserPersistence(directed=True, homology_dimensions=homology_dimensions)
FP.fit_transform_plot(directed_circle_fixed_point_gd);

## Where to next?

- [Topological feature extraction using `VietorisRipsPersistence` and `PersistenceEntropy`](https://giotto-ai.github.io/gtda-docs/latest/notebooks/vietoris_rips_quickstart.html) which treats the "special case" of point clouds (see below).
- Lorenz attractor.
- gtda.graphs and gtda.diagrams#
